# This implementation of the genetic algorithm randomly samples science missions and bins them

This is used to find the mean and standard deviatioin of the possible missions

Must be run on: python 3.12.0, SciPy 1.13.1, and numpy 2.0.1

In [ ]:
%reset
import numpy as np
import csv
from scipy.optimize import *
import sys
import matplotlib.pyplot as plt
np.set_printoptions(threshold=sys.maxsize)
import datetime
from scipy.stats import genpareto

# Loads in the analog payloads and the science objective scores

In [46]:
#import cell

#science
science_array = []

#take data from experiments csv and convert it to an array
with open('Found_Experiments.csv', 'rt') as f:
    array_y = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_y:
        science_array.append(line)
#print(science_array)


#take data from scores science csv and convert it to an array
objectives_score = []
with open('Objective_scores.csv', 'rt') as f:
    array_x = csv.reader(f, skipinitialspace=True, quotechar="'")
    for line in array_x:
        objectives_score.append(line)


# Prepares data from the .csv files and sets parameter values

In [ ]:
#data processing cell

#Parse objective data
science_array_array_mod = science_array[1:]                                         #cut off title score
science_mass_array      = np.asarray([i[-1] for i in science_array_array_mod],float)      #Masses of experiments 0 indexed
science_power_array     = np.asarray([i[-2] for i in science_array_array_mod],float)      #power consumption of experiments 0 indexed
objective_array_temp    = [np.array(i[1:-2]) for i in science_array_array_mod]  
objective_array         = []
for i in objective_array_temp:
    arr = []
    for j in i:
        s = j
        y = ""
        for char in s:
            if char.isdigit():
                y += char
        num = int(y)
        arr.append(num)
    
    objective_array.append(arr)
objective           = np.array(objectives_score[1:])
objective           = np.asarray(objective[0:, 1:],float)
objective_matrix    = np.zeros((len(objective_array),len(objective)))
for i in range(len(objective_matrix)):
    for j in objective_array[i]:
        objective_matrix[i][j-1] = 1

objective_matrix = np.transpose(objective_matrix) #matrix of shortfalls where the column are a tech and the value (0 or 1) represents of that index of a shortfall is completed

objective_Understanding = objective[0:, 1]
objective_Humans        = objective[0:, 2]

beta = 185 #kg/kW Power to mass ratio
max_mass    = 40000 #kg
min_mass    = 15000 #kg
mass_ratio  =  0.3 #kg

# The score function

In [48]:
#score function for science

def score_science(mission, check_mass=True):
    #weights
    w1 = 4  #completed_score
    w2 = 5  #understand_score
    w3 = 7  #human_score

    mass_max_science = max_mass*mass_ratio #in kg
    mass_min_science = min_mass*mass_ratio #in kg

    #Mass check
    if check_mass and (science_mass_array.dot(mission) + beta*science_power_array.dot(mission) <= mass_min_science or science_mass_array.dot(mission) + beta*science_power_array.dot(mission) > mass_max_science):
        return abs(science_mass_array.dot(mission) + beta*science_power_array.dot(mission))

    #normlizers
    max_number      = 46       #normalize number score 0-1
    max_understand  = 5        #normlize understanding score 0-1
    max_human       = 5        #normlize human score 0-1  

    #normlize weights to sum to 10
    W  = w1+w2+w3
    w1 = 10*w1/W
    w2 = 10*w2/W
    w3 = 10*w3/W

    mission                     = mission.astype(int) #Convete to ints
    completed                   = np.clip(objective_matrix @ mission, a_min=None, a_max=1) #A objective is either completed or not
    completed_number_score      = np.linalg.norm(completed)**2                             #Number of objective completed
    normalized_completed        = completed/completed_number_score                         #Normlizes completed array to weight each score

    completed_understanding_score   = objective_Understanding @ normalized_completed       #Generates the understanding score
    completed_human_score           = objective_Humans   @ normalized_completed            #Generates the human score

    #score calculation
    score = w1*completed_number_score/max_number + w2*completed_understanding_score/max_understand + w3*completed_human_score/max_human
    return -score


# Setup the number of bins

In [63]:
#setup
num_bins = 300
bins = np.zeros(num_bins)
bin_val = np.array([(10/num_bins) + 10*i/num_bins for i in range(num_bins)])
legth = len(science_array_array_mod)

# Loop through random comboniations

In [ ]:
#loop 3*10^6 10min 33s
for i in range(int(36*10**6)):
    mission = np.random.choice([0, 1], size=legth, p=[.8, .2])
    cur = -score_science(mission)
    for i in range(len(bin_val)-1):
        if cur <= bin_val[i] and cur > bin_val[i-1]:
            bins[i] += 1

# Save to a .csv file

In [71]:
#csv
now = datetime.datetime.now()
formatted_time = now.strftime("%d-%m-%Y_%H_%M_%S")
#write to csv
with open("bins_science" + formatted_time + '.csv', 'w', newline='') as save:
    wr = csv.writer(save)
    wr.writerow(bin_val)
    wr.writerow(bins)